## Matching two number tables

In [55]:
import pandas as pd

table1 = pd.DataFrame({
    'A': [2, 1],
    'B': [5, 2],
})


table2 = pd.DataFrame({
    'X': [2, 4, 1],
    'Y': [5, 7, 3]
})

In [56]:
# Initialize an empty list to store matching results
matching_results = []

# Loop through each row in table1
for _, row1 in table1.iterrows():
    # Initialize a list to store matching rows and their scores
    matches = []
    
    # Loop through each row in table2
    for _, row2 in table2.iterrows():
        
        # Count the number of matching columns
        match_score = sum(row1[col1] == row2[col2] for col1 in table1.columns for col2 in table2.columns)

        # If there's at least one matching column
        if match_score > 0:
            # Merge the dictionaries to form a new row containing all columns
            matched_row = {**row1, **row2}
            # Add match score to the new row
            matched_row['match_score'] = match_score
            # Append the matched row to matches
            matches.append(matched_row)
    
    # If there are matches for the current row in table1
    if matches:
        # Extend matching_results with matches
        matching_results.extend(matches)

# Create a DataFrame from the list of matching results
matching_results_df = pd.DataFrame(matching_results)
matching_results_df = matching_results_df.sort_values(by = ['A', 'B', 'match_score'], ascending=[True, True, False])

In [57]:
print("Table 1:")
print(table1)
print("\nTable 2:")
print(table2)
print("\nMatching Results:")
print(matching_results_df)

Table 1:
   A  B
0  2  5
1  1  2

Table 2:
   X  Y
0  2  5
1  4  7
2  1  3

Matching Results:
   A  B  X  Y  match_score
1  1  2  2  5            1
2  1  2  1  3            1
0  2  5  2  5            2


## Matching tables of numbers and strings 

In [58]:
# Sample DataFrames with mixed numeric and categorical columns
table1 = pd.DataFrame({
    'A': [1, 2],
    'B': ['cat', 'bird'],  
})

table2 = pd.DataFrame({
    'X': [1, 2, 1],
    'Y': ['dog', 'birdy', 'cat'], 
})

In [59]:
# Initialize an empty list to store matching results
matching_results = []

# Loop through each row in table1
for _, row1 in table1.iterrows():
    # Initialize a list to store matching rows and their scores
    matches = []
    
    # Loop through each row in table2
    for _, row2 in table2.iterrows():
        # Initialize match score
        match_score = 0
        
        # Check numeric columns for equality
        for col1, col2 in zip(table1.select_dtypes(include=['number']), table2.select_dtypes(include=['number'])):
            if row1[col1] == row2[col2]:
                match_score += 1
        
        # Check categorical columns for equality (convert to string for comparison)
        for col1, col2 in zip(table1.select_dtypes(exclude=['number']), table2.select_dtypes(exclude=['number'])):
            if str(row1[col1]) == str(row2[col2]):
                match_score += 1
        
        # If there's at least one matching column
        if match_score > 0:
            # Merge the dictionaries to form a new row containing all columns
            matched_row = {**row1, **row2}
            # Add match score to the new row
            matched_row['match_score'] = match_score
            # Append the matched row to matches
            matches.append(matched_row)
    
    # If there are matches for the current row in table1
    if matches:
        # Extend matching_results with matches
        matching_results.extend(matches)

# Create a DataFrame from the list of matching results
matching_results_df = pd.DataFrame(matching_results)
matching_results_df = matching_results_df.sort_values(by = ['A', 'B', 'match_score'], ascending=[True, True, False])

In [60]:
print("Table 1:")
print(table1)
print("\nTable 2:")
print(table2)
print("\nMatching Results:")
print(matching_results_df)

Table 1:
   A     B
0  1   cat
1  2  bird

Table 2:
   X      Y
0  1    dog
1  2  birdy
2  1    cat

Matching Results:
   A     B  X      Y  match_score
1  1   cat  1    cat            2
0  1   cat  1    dog            1
2  2  bird  2  birdy            1


## Introducing Fuzzy Match

In [61]:
# Sample DataFrames with mixed numeric and categorical columns
table1 = pd.DataFrame({
    'A': [1, 2],
    'B': ['cat', 'bird'],  
})

table2 = pd.DataFrame({
    'X': [1, 2, 1],
    'Y': ['dog', 'birdy', 'cat'], 
})

In [62]:
from fuzzywuzzy import fuzz  

# Initialize an empty list to store matching results
matching_results = []

# Loop through each row in table1
for _, row1 in table1.iterrows():
    # Initialize a list to store matching rows and their scores
    matches = []
    
    # Loop through each row in table2
    for _, row2 in table2.iterrows():
        # Initialize match score
        match_score = 0
        
        # Check numeric columns for equality
        for col1, col2 in zip(table1.select_dtypes(include=['number']), table2.select_dtypes(include=['number'])):
            if row1[col1] == row2[col2]:
                match_score += 1
        
        # Check string columns for similarity (fuzzy match)
        for col1, col2 in zip(table1.select_dtypes(exclude=['number']), table2.select_dtypes(exclude=['number'])):
            similarity_score = fuzz.ratio(str(row1[col1]), str(row2[col2]))
            if similarity_score >= 80:  # Adjust the threshold as needed
                match_score += 1
        
        # If there's at least one matching column
        if match_score > 0:
            # Merge the dictionaries to form a new row containing all columns
            matched_row = {**row1, **row2}
            # Add match score to the new row
            matched_row['match_score'] = match_score
            # Append the matched row to matches
            matches.append(matched_row)
    
    # If there are matches for the current row in table1
    if matches:
        # Extend matching_results with matches
        matching_results.extend(matches)

# Create a DataFrame from the list of matching results
matching_results_df = pd.DataFrame(matching_results)
matching_results_df = matching_results_df.sort_values(by = ['A', 'B', 'match_score'], ascending=[True, True, False])

In [63]:
print("Table 1:")
print(table1)
print("\nTable 2:")
print(table2)
print("\nMatching Results:")
print(matching_results_df)

Table 1:
   A     B
0  1   cat
1  2  bird

Table 2:
   X      Y
0  1    dog
1  2  birdy
2  1    cat

Matching Results:
   A     B  X      Y  match_score
1  1   cat  1    cat            2
0  1   cat  1    dog            1
2  2  bird  2  birdy            2
